
# Construction of Houly Time Series Data
The data for the motor vehicle collisions on public roads in Canada can be found in [open.canada.ca](https://open.canada.ca/data/en/dataset/1eb9eba7-71d1-4b30-9fb1-30cbdab7e63a).

In [1]:
# ignore deprecated warning
import warnings
warnings.filterwarnings("ignore")

# data manipulation
import pandas as pd
import numpy as np

# set font scale and style
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})

# Read raw data

In [2]:
base_path = 'python/ml-projects/imbalanced-learning'
df = pd.read_csv(base_path + '/canada-collision/data/clean_data.csv')

In [3]:
print("Data size:", df.shape)
df.columns = [x.lower() for x in df.columns]
df = df.dropna(subset = ['c_mnth']) # drop rows with missing values in c_mnth
df.head()

Data size: (6771768, 23)


,c_year,c_mnth,c_wday,c_hour,class,c_vehs,c_conf,c_rcfg,c_wthr,c_rsur,...,v_type,v_year,p_id,p_sex,p_age,p_psn,p_isev,p_safe,p_user,c_case
0,1999,January,Monday,20.0,0,2.0,Right turn,NaN,Clear and sunny,Icy,...,Other trucks and vans,1990.0,1.0,M,41.0,Driver,No Injury,NaN,Motor Vehicle Driver,752
1,1999,January,Monday,20.0,0,2.0,Right turn,NaN,Clear and sunny,Icy,...,Light Duty Vehicle,1987.0,1.0,M,19.0,Driver,No Injury,NaN,Motor Vehicle Driver,752
2,1999,January,Monday,20.0,0,2.0,Right turn,NaN,Clear and sunny,Icy,...,Light Duty Vehicle,1987.0,2.0,F,20.0,"Front row, right outboard",Injury,Safety device used,Motor Vehicle Passenger,752
3,1999,January,Monday,8.0,0,1.0,Hit a moving object,NaN,"Freezing rain, sleet, hail",Snow,...,Light Duty Vehicle,1986.0,1.0,M,46.0,Driver,No Injury,NaN,Motor Vehicle Driver,753
4,1999,January,Monday,8.0,0,1.0,Hit a moving object,NaN,"Freezing rain, sleet, hail",Snow,...,NaN,NaN,1.0,M,5.0,Pedestrian,Injury,NaN,Pedestrian,753


In [4]:
cols = ['c_year', 'c_mnth', 'c_wday', 'c_hour',]
cols_dict = {
    'class':'num_of_collisions', 
    'c_year':'year',
    'c_mnth':'month', 
    'c_wday':'weekday',  
    'c_hour':'hour',
}
pdf = pd.DataFrame(df.groupby(cols).count()['class']).reset_index()
print('Data size', pdf.shape)
pdf.rename(columns=cols_dict, inplace = True)
pdf.head()

Data size (38304, 5)


,year,month,weekday,hour,num_of_collisions
0,1999,April,Friday,0.0,56
1,1999,April,Friday,1.0,89
2,1999,April,Friday,2.0,75
3,1999,April,Friday,3.0,52
4,1999,April,Friday,4.0,32


# Extract more features

In [5]:
# change weekday to integer
wday_dict = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 
             'Friday':5,  'Saturday':6,   'Sunday':0
            }

# change weekday to workday
wrkday_dict = {1:True, 2:True, 3:True, 4:True, 
             5:True,  6:False, 0:False
            }

# change month to integer
month_dict = {'January':1, 'February':2, 'March':3, 
              'April':4,  'May':5, 'June':6,  'July':7,  
              'August':8, 'September':9,'October':10, 
              'November':11, 'December':12
             }

# change month to season
sea_dict = {1:'winter', 2:'winter', 3:'spring', 
              4:'spring',  5:'spring', 6:'summer', 
            7:'summer',  8:'summer', 9:'fall',
            10:'fall', 11:'fall', 12:'winter'
             }

pdf['weekday'].replace(wday_dict, inplace = True)
pdf['month'].replace(month_dict, inplace = True)
pdf['season'] = pdf['month'].replace(sea_dict)
pdf['workday'] = pdf['weekday'].replace(wrkday_dict)

In [12]:
cols = ['season', 'year', 'month','hour', 'weekday', 'workday', 'num_of_collisions']
df1 = pdf.copy()[cols]

group_cols = ['season', 'year', 'month','hour', 'weekday', 'workday']
final_df = pd.DataFrame(df1.groupby(group_cols).sum()['num_of_collisions']).reset_index()
final_df = final_df.sort_values(by = ['year','month','weekday'])

In [14]:
final_df.to_csv('../data/hourly_collision.csv', index = False)